In [7]:
import os
import numpy as np
import pandas as pd
import cv2
import json
import matplotlib.pyplot as plt
excel_path_270_z = '/hdd/sd2/data/TCT/biaozhu/dr.zhang/270.csv'
excel_path_270_xu = '/hdd/sd2/data/TCT/biaozhu/dr.zhang/徐医生270.csv'
excel_path_500_train = '/hdd/sd2/data/TCT/biaozhu/dr.zhang/500_train.csv'
excel_path_500_train_2 = '/hdd/sd2/data/TCT/biaozhu/dr.zhang/500_train2.csv'
excel_path_500_val = '/hdd/sd2/data/TCT/biaozhu/dr.zhang/500_val.csv'
data_path_500 = '/hdd/sd8/tlc/TCT/all_data/20190820/'
data_path_270 = '/hdd/sd8/tlc/TCT/all_data/20190717/'

In [2]:
df_270_z = pd.read_csv(excel_path_270_z)
df_270_xu = pd.read_csv(excel_path_270_xu)
df_500_train_1 = pd.read_csv(excel_path_500_train)
df_500_train_2 = pd.read_csv(excel_path_500_train_2)
df_500_val = pd.read_csv(excel_path_500_val)

In [23]:
df_270_z.head()

,用户手机号,用户姓名,影像结果编号,检查编号,序列编号,开始时间,提交时间,自定义内容,阴阳性,病灶,影像工具,影像结果类型,影像结果,恶性细胞分类(更新),TCT病灶类型
0,15618849360,张晴,497839,59f05b8ad569745386256b8c50c1b4ae3ea452f61,59f05b8ad569745386256b8c50c1b4ae3ea452f6,2019-09-25 16:56:45,2019-09-27 13:58:33,NaN,NaN,ANNO9,RECTANGLE,点位结果,"{""point1"":{""x"":1375.835390946502,""y"":435.35802...",4,2
1,15618849360,张晴,497838,59f05b8ad569745386256b8c50c1b4ae3ea452f61,59f05b8ad569745386256b8c50c1b4ae3ea452f6,2019-09-25 16:56:45,2019-09-27 13:58:33,NaN,NaN,ANNO9,RECTANGLE,点位结果,"{""point1"":{""x"":2.2057613168723336,""y"":487.9012...",4,2
2,15618849360,张晴,498059,59f05b8ad569745386256b8c50c1b4ae3ea452f61,59f05b8ad569745386256b8c50c1b4ae3ea452f6,2019-09-25 16:56:45,2019-09-27 13:58:33,NaN,NaN,ANNO9,RECTANGLE,点位结果,"{""point1"":{""x"":1130.771653543307,""y"":6.3832020...",4,2
3,15618849360,张晴,498422,59f05b8ad569745386256b8c50c1b4ae3ea452f61,59f05b8ad569745386256b8c50c1b4ae3ea452f6,2019-09-25 16:56:45,2019-09-27 13:58:33,NaN,NaN,ANNO9,RECTANGLE,点位结果,"{""point1"":{""x"":1231.6563120376766,""y"":1044.735...",3,1
4,15618849360,张晴,498424,59f05b8ad569745386256b8c50c1b4ae3ea452f61,59f05b8ad569745386256b8c50c1b4ae3ea452f6,2019-09-25 16:56:45,2019-09-27 13:58:33,NaN,NaN,ANNO9,RECTANGLE,点位结果,"{""point1"":{""x"":1596.0164609053497,""y"":220.1810...",2,1


In [3]:
df_500_train_1[u'TBS诊断'].value_counts()

2.0     347
4.0     270
0.0     260
7.0     145
5.0     120
8.0     109
3.0      68
10.0     48
6.0      24
Name: TBS诊断, dtype: int64

In [24]:
df_270_z[u'恶性细胞分类(更新)'].value_counts()

2     255
4     251
3     222
1     159
10     12
5       9
6       3
Name: 恶性细胞分类(更新), dtype: int64

In [44]:
annotations_train = []
annotations_val = []
map_z = {}
for i, row in df_500_train_1.iterrows():
    uid = row[u'序列编号']
    coord = json.loads(row[u'影像结果'])
    if pd.isnull(row[u'TBS诊断']):
        continue
    type_  = int(row[u'TBS诊断'])
    minx = int(coord['point1']['x'])
    miny = int(coord['point1']['y'])
    maxx = int(coord['point2']['x'])
    maxy = int(coord['point2']['y'])
    z = int(coord['point2']['z'])
    wsi_path = os.path.join(data_path_500, uid)
    image_pos = str(z)
    c = len(image_pos)
    img_name = ''
    for j in range(6-c):
        img_name += '0'
    img_name = img_name + image_pos + '.jpg'
    img_path = os.path.join(wsi_path, img_name)
    if type_ == 0:
        continue
    elif type_ < 10:
        type_ -= 2
    else:
        type_ = 7
    if img_path not in map_z:
        map_z[img_path] = [(minx, miny, maxx, maxy, type_)]
    else:
        map_z[img_path].append((minx, miny, maxx, maxy, type_))
        
for img_path in map_z:
    str_append = '\n' + img_path
    for coord in map_z[img_path]:
        xmin = coord[0]
        ymin = coord[1]
        xmax = coord[2]
        ymax = coord[3]
        type_ = coord[4]
        str_append += ' ' + str(xmin) + ',' + str(ymin) + ',' + str(xmax) + ',' + str(ymax) + ',' + str(type_)
    annotations_train.append(str_append)

map_z = {}
for i, row in df_500_train_2.iterrows():
    uid = row[u'序列编号']
    coord = json.loads(row[u'影像结果'])
    if pd.isnull(row[u'TBS诊断']):
        continue
    type_  = int(row[u'TBS诊断'])
    minx = int(coord['point1']['x'])
    miny = int(coord['point1']['y'])
    maxx = int(coord['point2']['x'])
    maxy = int(coord['point2']['y'])
    z = int(coord['point2']['z'])
    wsi_path = os.path.join(data_path_500, uid)
    image_pos = str(z)
    c = len(image_pos)
    img_name = ''
    for j in range(6-c):
        img_name += '0'
    img_name = img_name + image_pos + '.jpg'
    img_path = os.path.join(wsi_path, img_name)
    if type_ == 0:
        continue
    elif type_ < 10:
        type_ -= 2
    else:
        type_ = 7
    if img_path not in map_z:
        map_z[img_path] = [(minx, miny, maxx, maxy, type_)]
    else:
        map_z[img_path].append((minx, miny, maxx, maxy, type_))
        
for img_path in map_z:
    str_append = '\n' + img_path
    for coord in map_z[img_path]:
        xmin = coord[0]
        ymin = coord[1]
        xmax = coord[2]
        ymax = coord[3]
        type_ = coord[4]
        str_append += ' ' + str(xmin) + ',' + str(ymin) + ',' + str(xmax) + ',' + str(ymax) + ',' + str(type_)
    annotations_train.append(str_append)
    
map_z = {}    
for i, row in df_270_z.iterrows():
    uid = row[u'序列编号']
    coord = json.loads(row[u'影像结果'])
    if pd.isnull(row[u'恶性细胞分类(更新)']):
        continue
    type_  = int(row[u'恶性细胞分类(更新)'])
    minx = int(coord['point1']['x'])
    miny = int(coord['point1']['y'])
    maxx = int(coord['point2']['x'])
    maxy = int(coord['point2']['y'])
    z = int(coord['point2']['z'])
    wsi_path = os.path.join(data_path_270, uid)
    image_pos = str(z)
    c = len(image_pos)
    img_name = ''
    for j in range(6-c):
        img_name += '0'
    img_name = img_name + image_pos + '.jpg'
    img_path = os.path.join(wsi_path, img_name)
    if type_ == 0:
        continue
    elif type_ < 10:
        type_ -= 1
    else:
        continue
    if img_path not in map_z:
        map_z[img_path] = [(minx, miny, maxx, maxy, type_)]
    else:
        map_z[img_path].append((minx, miny, maxx, maxy, type_))
        
for img_path in map_z:
    str_append = '\n' + img_path
    for coord in map_z[img_path]:
        xmin = coord[0]
        ymin = coord[1]
        xmax = coord[2]
        ymax = coord[3]
        type_ = coord[4]
        str_append += ' ' + str(xmin) + ',' + str(ymin) + ',' + str(xmax) + ',' + str(ymax) + ',' + str(type_)
    annotations_train.append(str_append)
    
    
map_z = {}    
for i, row in df_500_val.iterrows():
    uid = row[u'序列编号']
    coord = json.loads(row[u'影像结果'])
    if pd.isnull(row[u'TBS诊断']):
        continue
    type_  = int(row[u'TBS诊断'])
    minx = int(coord['point1']['x'])
    miny = int(coord['point1']['y'])
    maxx = int(coord['point2']['x'])
    maxy = int(coord['point2']['y'])
    z = int(coord['point2']['z'])
    wsi_path = os.path.join(data_path_500, uid)
    image_pos = str(z)
    c = len(image_pos)
    img_name = ''
    for j in range(6-c):
        img_name += '0'
    img_name = img_name + image_pos + '.jpg'
    img_path = os.path.join(wsi_path, img_name)
    if type_ == 0:
        continue
    elif type_ < 10:
        type_ -= 2
    else:
        type_ = 7
    if img_path not in map_z:
        map_z[img_path] = [(minx, miny, maxx, maxy, type_)]
    else:
        map_z[img_path].append((minx, miny, maxx, maxy, type_))
        
for img_path in map_z:
    str_append = '\n' + img_path
    for coord in map_z[img_path]:
        xmin = coord[0]
        ymin = coord[1]
        xmax = coord[2]
        ymax = coord[3]
        type_ = coord[4]
        str_append += ' ' + str(xmin) + ',' + str(ymin) + ',' + str(xmax) + ',' + str(ymax) + ',' + str(type_)
    annotations_val.append(str_append)
    

In [45]:
len(annotations_train), len(annotations_val)

(2251, 455)

In [46]:
with open('./dataset/train_multi.txt', 'w') as f:
    for line in annotations_train:
        f.write(line)
        
with open('./dataset/val_multi.txt', 'w') as f:
    for line in annotations_val:
        f.write(line)        

In [51]:
folder_ = ['2019-08-21-112800#2', '2019-08-21-122118#2-1', '2019-08-21-124323#3-1', '2019-08-21-144433#2', '2019-08-21-153910#4'
         , '2019-08-21-161824#3', '2019-08-21-162511#1', '2019-08-21-163231#2', '2019-08-21-163444#4', '2019-08-21-164011#1', '2019-08-21-165622#4']

In [50]:
anno_csv = '/hdd/sd8/tlc/TCT/annotation/500_label.csv'
df = pd.read_csv(anno_csv)
df.head()

,uid,folder_name,label
0,001282923c7ba6eadcf8730936f356fb586278b0,2019-08-23-113228#3,1
1,017514326637623660441bbbd3d4d71002f811cd,2019-08-20-174738#4,0
2,01e50b01525e1425a3469c719a413229aee9ed90,2019-08-22-190741#2,1
3,024449ff10dd08c3bd2b64acbb83abf702dc1094,2019-08-21-174355#3,1
4,0245a9946d5281ca62306b655a47526424d368b8,2019-08-23-180208#3,0


In [52]:
for i in range(len(df)):
    uid = df['uid'].iloc[i]
    folder = df['folder_name'].iloc[i]
    if folder in folder_:
        print (uid)

10aeb0e63799f6e6da45be47308117d6981e2625
2c9ef020ed50efb5d4d771067797e411a491f9a6
3d9350b7a7866b0ce0ece511157f040e2c2f328f
493380774b484cab9ef7bb52905463fb853e3d40
63c3ae9792631082b2600c24b6d265789cf39ae0
757362b46a67685a5d1c8f320c4b4ae50d354017
772e288f06b30596879fe9ec969513859869057f
8735bc0ee2f5dda32176b18e8ce1633a9b24390c
bb9ec6525ab197aa0b279bd39e082de82c4a486b
bc155706751fe2db24424bf40993a396ba181f1f
f72825f63eb84bb840918e30d5c71bfc02e352a0


In [11]:
import os
from glob import glob

train_path = '/ssd2/yuyue/TCT_data/20200515_data/train/'
val_path = '/ssd2/yuyue/TCT_data/20200515_data/val/'


train_txt = './data_cls/train_0515.txt'
val_txt = './data_cls/val_0515.txt'
train_annotations = []
val_annotations = []

train_img_list = glob(train_path + '*/*/*/*.npy')
val_img_list = glob(val_path + '*/*/*/*.npy')

print (len(train_img_list), len(val_img_list))

for img in train_img_list:
    str1 = ''
    if 'neg' in img:
        str1 += '\n' + img + ' 0' 
    else:
        str1 += '\n' + img + ' 1'
    train_annotations.append(str1)
    
for img in val_img_list:
    str1 = ''
    if 'neg' in img:
        str1 += '\n' + img + ' 0' 
    else:
        str1 += '\n' + img + ' 1'
    val_annotations.append(str1)
        
print (len(train_annotations), len(val_annotations))

108034 14574
108034 14574


In [12]:
import numpy as np
np.random.shuffle(train_annotations)
np.random.shuffle(val_annotations)

In [13]:
#train_num = int(0.8*len(annotations))
with open(train_txt, 'w') as f:
    for line in train_annotations:
        f.write(line)
        
with open(val_txt, 'w') as f:
    for line in val_annotations:
        f.write(line)

In [14]:
len(train_annotations)

108034

In [26]:
train_txt = './data_cls/train_0515.txt'
val_txt = './data_cls/val_0515.txt'
train_pos_annotations = []
val_pos_annotations = []
train_neg_annotations = []
val_neg_annotations = []
with open(train_txt, 'r') as f:
    lines = f.readlines()
    for line in lines:
        if float(line.split(' ')[1]) == 1:
            train_pos_annotations.append(line.split(' ')[0])
        else:
            train_neg_annotations.append(line.split(' ')[0])

with open(val_txt, 'r') as f:
    lines = f.readlines()
    for line in lines:
        if float(line.split(' ')[1]) == 1:
            val_pos_annotations.append(line.split(' ')[0])
        else:
            val_neg_annotations.append(line.split(' ')[0])

In [29]:
import torch
torch.save(train_pos_annotations, './data_cls/train_pos.pth')
torch.save(train_neg_annotations, './data_cls/train_neg.pth')
torch.save(val_pos_annotations, './data_cls/val_pos.pth')
torch.save(val_neg_annotations, './data_cls/val_neg.pth')

In [33]:
import torch
val_dataset = torch.load('/ssd2/yuyue/TCT_data/5_fold_data/val.pth')
with open('./data_cls/val_0515.txt', 'w') as f:
    for line in val_dataset:
        img_path = line[0]
        label = line[1]
        if label > 0.5:
            label = '1'
        else:
            label = '0'
        str1 = img_path + ' ' + label + '\n'
        f.write(str1)

In [32]:
val_dataset

[('/ssd2/yuyue/TCT_data/20200515_data/val/other_neg/YFY_data/6_cut/21b0d9d90091c8550747f66686a3453e4c99dc54_000022_4.npy',
  0.05),
 ('/ssd2/yuyue/TCT_data/20200515_data/val/pos/CH_data/6_cut/89ff7bc60f082cf82de265b76891d0e746222e7d_000014_5.npy',
  0.95),
 ('/ssd2/yuyue/TCT_data/20200515_data/val/neg/YFY_data/random_cut/82db1e016b6b37bcf03ce2515aaf2b2a762f561b_000312.npy',
  0.05),
 ('/ssd2/yuyue/TCT_data/20200515_data/val/neg/CH_data/6_cut/d0d4d63f98af2b612b6fca53ace707300352da85_000430.npy',
  0.05),
 ('/ssd2/yuyue/TCT_data/20200515_data/val/pos/500_data/random_cut/578848b7cc40217f572bb71583e99bc0d5a282d2_000135_2.npy',
  0.95),
 ('/ssd2/yuyue/TCT_data/20200515_data/val/pos/YFY_data/random_cut/7f33f7a6050b03ec2d13bd656e59d45765c357d1_000152_3.npy',
  0.95),
 ('/ssd2/yuyue/TCT_data/TR_data/val/asc/TR_new_data/6_cut/80f4c820f7b8240f7de50430ddb58b44167dc9ca_000258_2.npy',
  0.95),
 ('/ssd2/yuyue/TCT_data/20200515_data/val/neg/500_data/random_cut/a990626c43ecb9638be433fd7d4a63d55c55050a